<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 6.  Vowpal Wabbit

На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных по посещению сайтов.

**План 6 недели:**
- Часть 1. Статья по Vowpal Wabbit
- Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов
 - 2.1. Подготовка данных
 - 2.2. Валидация по отложенной выборке
 - 2.3. Валидация по тестовой выборке (Public Leaderboard)

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. `sklearn.linear_model`. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
Также будет полезна [презентация](https://github.com/esokolov/ml-course-msu/blob/master/ML15/lecture-notes/Sem08_vw.pdf) лектора специализации Евгения Соколова. И, конечно же, [документация](https://github.com/JohnLangford/vowpal_wabbit/wiki) Vowpal Wabbit.

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите файл с ответами в соответствующем Programming Assignment. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1wteunpEhAt_9s-WBwxYphB6XpniXsAZiFSNuFNmvOdk).

## Часть 1. Статья про Vowpal Wabbit
Прочитайте [статью](https://habrahabr.ru/company/ods/blog/326418/) про Vowpal Wabbit на Хабре из серии открытого курса OpenDataScience по машинному обучению. Материал для этой статьи зародился из нашей специализации. Скачайте [тетрадку](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_russian/topic08_sgd_hashing_vowpal_wabbit/topic8_sgd_hashing_vowpal_wabbit.ipynb), прилагаемую к статье, посмотрите код, изучите его, поменяйте, только так можно разобраться с Vowpal Wabbit.

## Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов

### 2.1. Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные [отсюда](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.**

In [1]:
import os
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier
import pandas as pd
import numpy as np

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = ''

**Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке. **

In [3]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

In [4]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')

In [5]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

In [6]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

In [7]:
train_df_400['user_id'][:5]

session_id
1    653
2    198
3     34
4    601
5    273
Name: user_id, dtype: int64

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [8]:
from sklearn.preprocessing import LabelEncoder 

y = train_df_400['user_id'].values
class_encoder = LabelEncoder()
y_for_vw = class_encoder.fit_transform(y) + 1

In [9]:
print(y.shape, y_for_vw.shape)

(182793,) (182793,)


**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**
- объедините обучающиую и тестовую выборки
- выберите только сайты (признаки от 'site1' до 'site10')
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат `csr_matrix`
- разбейте обратно на обучающую и тестовую части

In [10]:
sites = ['site' + str(i) for i in range(1, 11)]

In [131]:
from sklearn.feature_extraction.text import CountVectorizer

train_test_df = pd.concat([train_df_400, test_df_400], sort=False)
train_test_df_sites = train_test_df[sites].fillna(0).astype('int')

vectorizer = CountVectorizer(stop_words=['0'], token_pattern="\d+")
X = vectorizer.fit_transform(train_test_df_sites.to_string(index=False, header=False).splitlines())
print(X.shape)

X_train_sparse = X[:len(train_df_400)]
X_test_sparse = X[len(train_df_400):]
#y = ''' ВАШ КОД ЗДЕСЬ '''
print(X_train_sparse.shape, X_test_sparse.shape)

(229266, 36656)
(182793, 36656) (46473, 36656)


In [132]:
np.unique(train_test_df_sites.values)

array([    0,     1,     2, ..., 36654, 36655, 36656])

In [126]:
v = CountVectorizer(ngram_range=(1, 2), stop_words=['0'], token_pattern="\d+")
#print(v.fit(["fox dog fox fox dog fox fox fox fox fox"]).stop_words_)
print(v.fit(["1 2 1 1 2 1 1 1 1 1", "1 2 0 0 0 0 0 0 1 1 1"]).vocabulary_)
print(v.fit_transform(["1 2 1 1 2 1 1 1 1 1", "1 2 0 0 0 0 0 0 1 1 1"]).A)

{'1': 0, '2': 3, '1 2': 2, '2 1': 4, '1 1': 1}
[[8 5 2 2 2]
 [4 2 1 1 1]]


In [133]:
print(X_train_sparse.shape, X_test_sparse.shape)

(182793, 36656) (46473, 36656)


### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [134]:
train_share = int(.7 * train_df_400.shape[0])

train_df_part = train_df_400[sites].iloc[:train_share, :]
valid_df = train_df_400[sites].iloc[train_share:, :]

X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [135]:
y_train_part = y[:train_share]
y_valid = y[train_share:]

y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

**Реализуйте функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.**

Вход:
 - X – матрица `NumPy` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [152]:
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw'):
    with open(out_file, mode='w', encoding='UTF-8') as file:        
        for index, row in enumerate(X):
            file.write('1' if y is None else str(y[index]))
            file.write(' |s ')
            file.write(" ".join([str(item) for item in row if item != 0]))
            file.write('\n')

**Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора `NumPy`.**

In [153]:
%%time
# будет 4 вызова
arrays_to_vw(train_df_part.fillna(0).astype('int').values, y_train_part_for_vw, train=True, out_file='train_part.vw')
arrays_to_vw(valid_df.fillna(0).astype('int').values, y_valid_for_vw, train=False, out_file='valid.vw')
arrays_to_vw(train_df_400[sites].fillna(0).astype('int').values, y_for_vw, train=True, out_file='train.vw')
arrays_to_vw(test_df_400[sites].fillna(0).astype('int').values, y=None, train=False, out_file='test.vw')

Wall time: 5.85 s


**Результат должен получиться таким.**

In [7]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [8]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [9]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


**Обучите модель Vowpal Wabbitна выборке `train_part.vw`. Укажите, что решается задача классификации с 400 классами (`--oaa`), сделайте 3 прохода по выборке (`--passes`). Задайте некоторый кэш-файл (`--cache_file`, можно просто указать флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажите значение параметра `b`=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите `random_seed`=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [154]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

**Запишите прогнозы на выборке *valid.vw* в *vw_valid_pred.csv*.**

In [155]:
%%time
!vw -d train_part.vw --oaa 400 --passes 3 -c -k -b 26 --random_seed 17 -f train_part_model.vw

^C
Wall time: 2min 54s


In [150]:
!vw -i train_part_model.vw -c -k --random_seed 17 -t -d valid.vw -p vw_valid_pred.csv

Generating 0-grams for s namespaces.
only testing
predictions = vw_valid_pred.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = valid.vw.cache
Reading datafile = valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4      188       11
1.000000 1.000000            2            2.0      160      220       11
0.750000 0.500000            4            4.0      143      143       11
0.750000 0.750000            8            8.0      247      247       11
0.750000 0.750000           16           16.0      341       30        4
0.625000 0.500000           32           32.0      237      237       11
0.625000 0.625000           64           64.0      178      178       11
0.648438 0.671875          128          128.0      132      228       11
0.667969 0.687500          256        

**Считайте прогнозы *kaggle_data/vw_valid_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [151]:
from sklearn.metrics import accuracy_score
with open('vw_valid_pred.csv') as pred_file:
    pred_user = [float(label) for label in pred_file.readlines()]
vw_valid_acc = round(accuracy_score(y_valid_for_vw, pred_user), 3)    
vw_valid_acc

0.347

**Теперь обучите `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделайте прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитайте доли верных ответов. Логистическая регрессия будет обучаться не быстро (у меня – 4 минуты) – это нормально. Укажите везде `random_state`=17, `n_jobs`=-1. Для `SGDClassifier` также укажите `max_iter=3`.**

In [41]:
logit = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(max_iter=3, random_state=17, n_jobs=-1, loss='log')

In [46]:
%%time
logit.fit(X_train_part_sparse, y_train_part)

E:\work\tools\python\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 27min 10s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [42]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)

Wall time: 18.2 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=3, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

**<font color='red'>Вопрос 1. </font> Посчитайте долю правильных ответов на отложенной выборке для Vowpal Wabbit, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 2. </font> Посчитайте долю правильных ответов на отложенной выборке для SGD, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 3. </font> Посчитайте долю правильных ответов на отложенной выборке для логистической регрессии, округлите до 3 знаков после запятой.**

In [47]:
vw_valid_acc = round(accuracy_score(y_valid_for_vw, pred_user), 3)
sgd_valid_acc = round(accuracy_score(y_valid, sgd_logit.predict(X_valid_sparse)), 3)
logit_valid_acc = round(accuracy_score(y_valid, logit.predict(X_valid_sparse)), 3)

In [48]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [49]:
write_answer_to_file(round(vw_valid_acc, 3), 'answer6_1.txt')
write_answer_to_file(round(sgd_valid_acc, 3), 'answer6_2.txt')
write_answer_to_file(round(logit_valid_acc, 3), 'answer6_3.txt')

### 2.3. Валидация по тестовой выборке (Public Leaderboard)

**Обучите модель VW с теми же параметрами на всей обучающей выборке – *train.vw*.**

In [161]:
%%time
!vw -d train.vw --oaa 400 -c -k -b 26 --random_seed 17 -f train_model.vw

Wall time: 55.2 s


Generating 2-grams for s namespaces.
Generating 1-skips for s namespaces.
final_regressor = train_model.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = train.vw.cache
Reading datafile = train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       28
1.000000 1.000000            2            2.0       82      262       28
1.000000 1.000000            4            4.0      241      262       28
1.000000 1.000000            8            8.0      352      262       10
1.000000 1.000000           16           16.0      156      176       28
1.000000 1.000000           32           32.0      243      166       28
0.984375 0.968750           64           64.0       79      156       28
0.984375 0.984375          128          128.0      298       99       19
0.957031 0.92

**Сделайте прогноз для тестовой выборки.**

In [162]:
%%time
!vw -i train_model.vw -c -k --random_seed 17 -t -d test.vw -p vw_test_pred.csv

Wall time: 2.64 s


Generating 2-grams for s namespaces.
Generating 1-skips for s namespaces.
only testing
predictions = vw_test_pred.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = test.vw.cache
Reading datafile = test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        1      310       28
1.000000 1.000000            2            2.0        1       21       28
1.000000 1.000000            4            4.0        1      265       28
1.000000 1.000000            8            8.0        1      310       28
1.000000 1.000000           16           16.0        1      349       28
1.000000 1.000000           32           32.0        1      388       28
1.000000 1.000000           64           64.0        1      139       28
1.000000 1.000000          128          128.0        1      386        2
1.00

**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.**

In [163]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [164]:
from sklearn.metrics import accuracy_score
with open('vw_test_pred.csv') as pred_file:
    pred_user = [int(label) - 1 for label in pred_file.readlines()]

vw_pred = class_encoder.inverse_transform(pred_user)
vw_pred

E:\work\tools\python\anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([789,  48, 795, ..., 656, 478, 179], dtype=int64)

In [165]:
write_to_submission_file(vw_pred, os.path.join(PATH_TO_DATA, 'vw_400_users.csv'))

In [61]:
write_answer_to_file(0.18768, 'answer6_4.txt')

**Сделайте то же самое для SGD и логистической регрессии. Тут уже ждать обучение логистической регрессии совсем скучно (заново запускать тетрадку вам не захочется), но давайте дождемся.**

In [62]:
logit = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(max_iter=3, random_state=17, n_jobs=-1, loss='log')

In [66]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)

Wall time: 14.8 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=3, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [70]:
%%time
logit.fit(X_train_sparse, y)

E:\work\tools\python\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 45min 18s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [67]:
write_to_submission_file(sgd_logit.predict(X_test_sparse), os.path.join(PATH_TO_DATA, 'sgd_400_users.csv'))

In [71]:
write_to_submission_file(logit.predict(X_test_sparse), os.path.join(PATH_TO_DATA, 'logit_400_users.csv'))

In [69]:
write_answer_to_file(0.17985, 'answer6_5.txt')

In [72]:
write_answer_to_file(0.19616, 'answer6_6.txt')

In [ ]:
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'logit_400_users.csv'))
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'sgd_400_users.csv'))

Посмотрим на доли правильных ответов на публичной части (public leaderboard) тестовой выборки [этого](https://inclass.kaggle.com/c/identify-me-if-you-can4) соревнования.

**<font color='red'>Вопрос 4. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для Vowpal Wabbit?**

**<font color='red'>Вопрос 5. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для SGD?**

**<font color='red'>Вопрос 6. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для логистической регрессии?**


In [ ]:
vw_lb_score, sgd_lb_score, logit_lb_score = ''' ВАШ КОД ЗДЕСЬ '''

write_answer_to_file(round(vw_lb_score, 3), 'answer6_4.txt')
write_answer_to_file(round(sgd_lb_score, 3), 'answer6_5.txt')
write_answer_to_file(round(logit_lb_score, 3), 'answer6_6.txt')

**В заключение по заданию:**
- Про соотношение качества классификации и скорости обучения VW, SGD и logit выводы предлагается сделать самостоятельно
- Пожалуй, задача классификации на 400 классов (идентификация 400 пользователей) решается недостаточно хорошо при честном отделении по времени тестовой выборки от обучающей. Далее мы будем соревноваться в идентификации одного пользователя (Элис) – [вот](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнование, в котором предлагается поучаствовать. Не перепутайте! 

**Удачи!**